In [4]:
import pandas as pd
import pbipy
import requests
from azure.identity import ClientSecretCredential
import json

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

In [3]:
tenant_id = os.getenv('tenant_id')
client_id = os.getenv('client_id')
client_secret = os.getenv('client_secret')

In [1]:
from invoke_fabric_api import *

In [2]:
workspace_id = 'bed78b83-ac16-4adc-a65a-257c9b99a3f6'
lakehouse_id = 'c19671a8-29e7-4ed2-8570-78541d714684'

In [48]:
import requests

# Define the parameters
tenant = os.getenv('tenant_id')  # Replace with your tenant ID
client_id = os.getenv('client_id')
redirect_uri = 'http://localhost/myapp/'
response_type = 'code'
response_mode = 'query'
# scope = ['https://api.fabric.microsoft.com/OneLake.ReadWrite.All']
scope= 'offline_access https://api.fabric.microsoft.com/OneLake.ReadWrite.All'
state = '12345'

# Construct the URL
url = f'https://login.microsoftonline.com/{tenant}/oauth2/v2.0/authorize'

# Create the query parameters
params = {
    'client_id': client_id,
    'response_type': response_type,
    'redirect_uri': redirect_uri,
    'response_mode': response_mode,
    'scope': scope,
    'state': state
}

# Make the GET request
response = requests.get(url, params=params)

# Print the response URL
print(response.url)

https://login.microsoftonline.com/5cd64994-64c0-4c59-bca6-801527b1a508/oauth2/v2.0/authorize?client_id=80c581f3-02d7-4bbf-b371-8a61b80740ba&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%2Fmyapp%2F&response_mode=query&scope=offline_access+https%3A%2F%2Fapi.fabric.microsoft.com%2FOneLake.ReadWrite.All&state=12345


In [49]:
code='0.AcYAlEnWXMBkWUy8poAVJ7GlCPOBxYDXAr9Ls3GKYbgHQLrGAK0.AgABBAIAAADW6jl31mB3T7ugrWTT8pFeAwDs_wUA9P_-YETIa6Voynuie7EtrAUwXUbv-MrXp0IoVzLNrAiFrBSUTTlYhFgHQ24a5stgigTk0b0zrS5tcLuu_FwvesOZDxJXr38E18eHB5uPP7sUnOvCgRYgzssy6aSKziL02INSILgqlZAkY_N6YASS_V8y9QAQ3bKbMP93sYsFKsfP4w_zb7GSUTcol5S73S9LKrENMPtHtB-tJyc1dNXwFxNekECFsSn1tUi8a8sgSoRiLa5Hwh-ncL_yljRoKgy5_XNC8CuBVCw_nLqN4kWzoGUkx0080k1QWVHtyS1yZDD1XyrhaC5QZX-Ov-SsSolzTlFEyhMEdylzn39eIEI5Wqow2WJ_Kqa9eP74jUgKxNA9Mf-kEs_E_Cj9w_s-kr7aWyRcbFLNCxcyaX3ldm_wsyQDceJnTUWwUiAxWy1m1OuT5fmh8KHN6HDiYoEGiu62gVR_aFkTZgY_U9tttmRNOKyA6gxSRvjKNLO4jZmkdiu783WtEfdZ-DsWrDXwwcFFpmI5GSisBGQAf7OrosOmMwWgqUAPWuAM6elbeGMP_Hx77cubq-O34N_emmLnmHi9SEkLsq80eSpuVZ3Wl6Q2yVGCStVU2TKS5PftAMC6xMZkispPkaARirV6PcM2pzAGsojYaS_LjSJdox5YUl2oR4xvHU8iQeX8KtYx44khQYarna8dQG0ycbGFR8XQl5MO_UZmjAFdPiLVPaa2HuLclYeQhG4s6X2YHZEZmoyzBpY7LVF47GoxJfYboZE'

In [23]:
code

'0.AcYAlEnWXMBkWUy8poAVJ7GlCPOBxYDXAr9Ls3GKYbgHQLrGAK0.AgABBAIAAADW6jl31mB3T7ugrWTT8pFeAwDs_wUA9P9X9qFElbTspxMjvYaHBQlm61DdzpJ7sEeFISAKE6v6fiz5OHftHhikET2mIJfok-n9G2yjdBzWCVug2Ux__njRVyERcW6zHwylkaojhPR7k-Nc1SM48riMZhzoljli2cIpT87kYRNfTXHJybmp-zmLWcXB7gPwLcJqdDB4gAi8Lq0c85kckCSJEF4iC4Oyd1VeT4Ni1od-KFGK9S5BZdEbwtoBfOmKAr8NY0U989l9ExRMGMSU9VN59VAc32igmE50fEy3zrCFb6ranccUuuQvLtElJruuaR3EaXaPHLlhX0F2RdPjmAZXTZ5PPps-BHMdzduw8vLtFZa1G4cOo1spO8yRAuLym0fqDe-rlql9nd1W6-b2pjlwIEoGVgL2ptW5wrAkDt-X1597KB8pyZ8EWXEztre6dIqLZvFI2EFoXimwnaaHP5aKPzfeTF89ltd2Lich3ZrpstpVsianTOFKLlvyjwRdXDbAYVuY4-8JdFVIR6uHt12IV3-MnG-4_oiTPz0fOZSSxb72rkjCk5wfg3sCNWxDdH4l8fnpP1ipRbYkYaknRXRGODOKEsPg6G32Y3rpeGKoZretI4kUjdxtTH3ywwn44xpbXwzz8-sILFFh2UQGuAgX-z64AFodx1xOsp9VlHSFRghXgwLPmsi7Dbyc39yTC8Df14hm3MKk0ESLiYwZVmSPypXwu9inpWd8xP7TNp32snI&state=12345&session_state=7f6cd68f-b260-406e-a86e-69d612adb87a#'

In [50]:
url = f'https://login.microsoftonline.com/{tenant}/oauth2/v2.0/token'

# Prepare the data for the request
data = {
    'client_id': os.getenv('client_id'),
    'scope': scope,
    'code': code,
    'redirect_uri': 'http://localhost/myapp/',
    'grant_type': 'authorization_code',
    'client_secret': os.getenv('client_secret')  # Only required for web apps
}

# Make the POST request
response = requests.post(url, data=data)

# Check the response
if response.status_code == 200:
    token_info = response.json()
    print('Access Token:', token_info['access_token'])
else:
    print('Error:', response.status_code, response.text)


Access Token: eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6IjNQYUs0RWZ5Qk5RdTNDdGpZc2EzWW1oUTVFMCIsImtpZCI6IjNQYUs0RWZ5Qk5RdTNDdGpZc2EzWW1oUTVFMCJ9.eyJhdWQiOiJodHRwczovL2FwaS5mYWJyaWMubWljcm9zb2Z0LmNvbSIsImlzcyI6Imh0dHBzOi8vc3RzLndpbmRvd3MubmV0LzVjZDY0OTk0LTY0YzAtNGM1OS1iY2E2LTgwMTUyN2IxYTUwOC8iLCJpYXQiOjE3Mjk3NTkyOTMsIm5iZiI6MTcyOTc1OTI5MywiZXhwIjoxNzI5NzYzMjQxLCJhY2N0IjowLCJhY3IiOiIxIiwiYWlvIjoiQVZRQXEvOFlBQUFBZVgrTjdNLzVXZTRRRXBSVENjOEdvd005cU0xMmIzd29oQ2tvcnd2aGpJeUg0VmV6eTRwMkx4d1U4Y2k0SmwwdUxKVHprN2k3cUErSFYyNFJFOUl2L2RQK3Y1TS9jOXpHMHMvS01VNVk1T1E9IiwiYW1yIjpbInB3ZCIsIm1mYSJdLCJhcHBpZCI6IjgwYzU4MWYzLTAyZDctNGJiZi1iMzcxLThhNjFiODA3NDBiYSIsImFwcGlkYWNyIjoiMSIsImZhbWlseV9uYW1lIjoiTEUiLCJnaXZlbl9uYW1lIjoiVFkiLCJpZHR5cCI6InVzZXIiLCJpcGFkZHIiOiIxMTMuMTYxLjM5LjE0MyIsIm5hbWUiOiJUWSBMRSIsIm9pZCI6ImRjOGU0MjBmLWIwNDYtNGFhMy1hNzVjLWNiMmI0OTY1YmIzYyIsInB1aWQiOiIxMDAzMjAwM0IyRTc4RkQyIiwicmgiOiIwLkFjWUFsRW5XWE1Ca1dVeThwb0FWSjdHbENBa0FBQUFBQUFBQXdBQUFBQUFBQUFER0FLMC4iLCJzY3AiOiJPbmVMYWtlLlJlYWRXc

In [52]:
method = "post"
shortcut_conflict_policy = "GenerateUniqueName"
uri = f"workspaces/{workspace_id}/items/{lakehouse_id}/shortcuts?shortcutConflictPolicy={shortcut_conflict_policy}"

ws_from = 'bed78b83-ac16-4adc-a65a-257c9b99a3f6'
lk_from = '76dfbe3f-19e2-4221-a465-d3458852309c'

payload = {
  "path": "Tables",
  "name": "onelake_tables_sc",
  "target": {
     "oneLake": {
      "workspaceId": ws_from,
      "itemId": lk_from,
      "path": "Tables/test2"
      }
  }
}

API_ENDPOINT = "api.fabric.microsoft.com/v1"



headers = {
    "Authorization": "Bearer " + token_info['access_token'],
    "Content-Type": "application/json"
}

try:
    url = f"https://{API_ENDPOINT}/{uri}"
        
    session = requests.Session()
    retries = Retry(total=3, backoff_factor=5, status_forcelist=[502, 503, 504])
    adapter = HTTPAdapter(max_retries=retries)
    session.mount('http://', adapter)
    session.mount('https://', adapter)

    response = session.request(method, url, headers=headers, json=payload, timeout=240)

    response_details = {
        'status_code': response.status_code,
        'response': response.json() if response.content else {},
        'headers': dict(response.headers) 
    }
    print(json.dumps(response_details, indent=2))

except requests.RequestException as ex:
    print(ex)

{
  "status_code": 403,
  "response": {
    "requestId": "ac2356a5-8b3b-4824-a5e4-c86b03098422",
    "errorCode": "InsufficientPrivileges",
    "moreDetails": [
      {
        "errorCode": "Forbidden",
        "message": "User is not authorized to perform current operation for workspace 'bed78b83-ac16-4adc-a65a-257c9b99a3f6', item 'c19671a8-29e7-4ed2-8570-78541d714684'."
      }
    ],
    "message": "The caller does not have sufficient permissions to access the requested resource"
  },
  "headers": {
    "Cache-Control": "no-store, must-revalidate, no-cache",
    "Pragma": "no-cache",
    "Transfer-Encoding": "chunked",
    "Content-Type": "application/json; charset=utf-8",
    "x-ms-public-api-error-code": "InsufficientPrivileges",
    "Strict-Transport-Security": "max-age=31536000; includeSubDomains",
    "X-Frame-Options": "deny",
    "X-Content-Type-Options": "nosniff",
    "RequestId": "ac2356a5-8b3b-4824-a5e4-c86b03098422",
    "Access-Control-Expose-Headers": "RequestId",
    

In [51]:
response.json

{'token_type': 'Bearer',
 'scope': 'https://api.fabric.microsoft.com/OneLake.ReadWrite.All',
 'expires_in': 3647,
 'ext_expires_in': 3647,
 'access_token': 'eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6IjNQYUs0RWZ5Qk5RdTNDdGpZc2EzWW1oUTVFMCIsImtpZCI6IjNQYUs0RWZ5Qk5RdTNDdGpZc2EzWW1oUTVFMCJ9.eyJhdWQiOiJodHRwczovL2FwaS5mYWJyaWMubWljcm9zb2Z0LmNvbSIsImlzcyI6Imh0dHBzOi8vc3RzLndpbmRvd3MubmV0LzVjZDY0OTk0LTY0YzAtNGM1OS1iY2E2LTgwMTUyN2IxYTUwOC8iLCJpYXQiOjE3Mjk3NTkyOTMsIm5iZiI6MTcyOTc1OTI5MywiZXhwIjoxNzI5NzYzMjQxLCJhY2N0IjowLCJhY3IiOiIxIiwiYWlvIjoiQVZRQXEvOFlBQUFBZVgrTjdNLzVXZTRRRXBSVENjOEdvd005cU0xMmIzd29oQ2tvcnd2aGpJeUg0VmV6eTRwMkx4d1U4Y2k0SmwwdUxKVHprN2k3cUErSFYyNFJFOUl2L2RQK3Y1TS9jOXpHMHMvS01VNVk1T1E9IiwiYW1yIjpbInB3ZCIsIm1mYSJdLCJhcHBpZCI6IjgwYzU4MWYzLTAyZDctNGJiZi1iMzcxLThhNjFiODA3NDBiYSIsImFwcGlkYWNyIjoiMSIsImZhbWlseV9uYW1lIjoiTEUiLCJnaXZlbl9uYW1lIjoiVFkiLCJpZHR5cCI6InVzZXIiLCJpcGFkZHIiOiIxMTMuMTYxLjM5LjE0MyIsIm5hbWUiOiJUWSBMRSIsIm9pZCI6ImRjOGU0MjBmLWIwNDYtNGFhMy1hNzVjLWNiMmI0OTY1YmIzYyIsIn

In [34]:
tenant = '{tenant}'  # Replace with your tenant ID
url = f'https://login.microsoftonline.com/{tenant}/oauth2/v2.0/token'

# Set the data for the POST request
data = {
    'client_id': '11111111-1111-1111-1111-111111111111',
    'scope': 'user.read mail.read',
    'refresh_token': 'OAAABAAAAiL9Kn2Z27UubvWFPbm0gLWQJVzCTE9UkP3pSx1aXxUjq...',  # Replace with your refresh token
    'grant_type': 'refresh_token',
    'client_secret': 'jXoM3iz...'  # Replace with your client secret if required
}

# Make the POST request
response = requests.post(url, data=data)

# Print the response
print(response.status_code)
print(response.json())  # Print the JSON response, if applicable

{'token_type': 'Bearer',
 'scope': 'https://api.fabric.microsoft.com/OneLake.ReadWrite.All',
 'expires_in': 4067,
 'ext_expires_in': 4067,
 'access_token': 'eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6IjNQYUs0RWZ5Qk5RdTNDdGpZc2EzWW1oUTVFMCIsImtpZCI6IjNQYUs0RWZ5Qk5RdTNDdGpZc2EzWW1oUTVFMCJ9.eyJhdWQiOiJodHRwczovL2FwaS5mYWJyaWMubWljcm9zb2Z0LmNvbSIsImlzcyI6Imh0dHBzOi8vc3RzLndpbmRvd3MubmV0LzVjZDY0OTk0LTY0YzAtNGM1OS1iY2E2LTgwMTUyN2IxYTUwOC8iLCJpYXQiOjE3Mjk3NTg3NzEsIm5iZiI6MTcyOTc1ODc3MSwiZXhwIjoxNzI5NzYzMTM5LCJhY2N0IjowLCJhY3IiOiIxIiwiYWlvIjoiQVZRQXEvOFlBQUFBdkxUemp1SGxBUk0yeGlLR2JGMmplZSt5RGlOalhNcEdjVk9OUGJJd1VlMWhHd0VCNEExRHJ1c01ySEdGcTBtbEU3YUVSaUZFaVVhWGI0U3I3Unl3RitJVnFxbXUzc1o1RjRVMUF6YXlIZms9IiwiYW1yIjpbInB3ZCIsIm1mYSJdLCJhcHBpZCI6IjgwYzU4MWYzLTAyZDctNGJiZi1iMzcxLThhNjFiODA3NDBiYSIsImFwcGlkYWNyIjoiMSIsImZhbWlseV9uYW1lIjoiTEUiLCJnaXZlbl9uYW1lIjoiVFkiLCJpZHR5cCI6InVzZXIiLCJpcGFkZHIiOiIxMTMuMTYxLjM5LjE0MyIsIm5hbWUiOiJUWSBMRSIsIm9pZCI6ImRjOGU0MjBmLWIwNDYtNGFhMy1hNzVjLWNiMmI0OTY1YmIzYyIsIn

In [17]:
method = "post"
shortcut_conflict_policy = "GenerateUniqueName"
uri = f"workspaces/{workspace_id}/items/{lakehouse_id}/shortcuts?shortcutConflictPolicy={shortcut_conflict_policy}"

ws_from = 'bed78b83-ac16-4adc-a65a-257c9b99a3f6'
lk_from = '76dfbe3f-19e2-4221-a465-d3458852309c'

payload = {
  "path": "Tables",
  "name": "onelake_tables_sc",
  "target": {
     "oneLake": {
      "workspaceId": ws_from,
      "itemId": lk_from,
      "path": "Tables/test2"
      }
  }
}

invoke_fabric_api_request(method, uri, payload)

{
  "status_code": 403,
  "response": {
    "requestId": "05797f22-cd50-402b-858f-692e21202a27",
    "errorCode": "InsufficientPrivileges",
    "moreDetails": [
      {
        "errorCode": "Forbidden",
        "message": "User is not authorized to perform current operation for workspace 'bed78b83-ac16-4adc-a65a-257c9b99a3f6', item 'c19671a8-29e7-4ed2-8570-78541d714684'."
      }
    ],
    "message": "The caller does not have sufficient permissions to access the requested resource"
  },
  "headers": {
    "Cache-Control": "no-store, must-revalidate, no-cache",
    "Pragma": "no-cache",
    "Transfer-Encoding": "chunked",
    "Content-Type": "application/json; charset=utf-8",
    "x-ms-public-api-error-code": "InsufficientPrivileges",
    "Strict-Transport-Security": "max-age=31536000; includeSubDomains",
    "X-Frame-Options": "deny",
    "X-Content-Type-Options": "nosniff",
    "RequestId": "05797f22-cd50-402b-858f-692e21202a27",
    "Access-Control-Expose-Headers": "RequestId",
    

In [6]:
workspace_name = 'dev-test-gj'

In [7]:
pbi_api = 'https://analysis.windows.net/powerbi/api/.default'

In [8]:
auth = ClientSecretCredential(authority = 'https://login.microsoftonline.com/',
                                            tenant_id = tenant_id,
                                            client_id = client_id,
                                            client_secret = client_secret)
access_token = auth.get_token(pbi_api)
access_token = access_token.token

In [10]:
base_url = 'https://api.powerbi.com/v1.0/myorg/'
header = {'Authorization': f'Bearer {access_token}',
          'Content-Type': 'application/json'}

In [11]:
pbi = pbipy.PowerBI(access_token)

In [12]:
admin = pbi.admin()

In [13]:
def check_workspace_exists(df, workspace_name):
    existing_name = set(df['name'].values)
    result = workspace_name in existing_name
    return result

In [14]:
workspace_tenant = admin.groups()

In [16]:
data = [{'Group id': ws.id, 'name': ws.name} for ws in workspace_tenant]

In [18]:
df_workspace = pd.DataFrame(data)

In [20]:
workspace_check_existence = check_workspace_exists(df_workspace, workspace_name)

In [ ]:
if not workspace_check_existence:
    # Create DEV workspace
    workspace_dev = pbi.create_group(name = workspace_name)
    print(f"Workspace '{workspace_dev.name}' created.")
    # Create TEST workspace
    workspace_test = pbi.create_group(name = workspace_name.replace('dev', 'test'))
    print(f"Workspace '{workspace_test.name}' created.")
    # Create PROD workspace
    workspace_prod = pbi.create_group(name = workspace_name.replace('dev', 'prod'))
    print(f"Workspace '{workspace_prod.name}' created.")
else:
    print(f"A workspace with the name '{workspace_name}' already exists")